In [ ]:
# Package
import pandas as pd 

In [ ]:
# Read the movies.csv file
movie_base=pd.read_csv("movies.csv")

In [ ]:
#let's create a function to get the reviewers and their ratings 
def collect_reviewers_for_movies(dataframe, total_reviews_per_movie=10):
    # Initialize WebDriver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(5)
    
    # Create a dictionary to store the collected data
    collected_data = {
        "Movie_Title": [],
        "Rating": [],
        "User_Name": []
    }
    # Iterate over the rows in the dataframe
    for index, row in dataframe.iterrows():
        url = row['link_review'] 

        page = 1
        collected_reviews = 0

        # Load the movie's reviews page
        driver.get(url) 
        # Get the movie title from the dataframe
        movie_title = row['Title']  

        while collected_reviews < total_reviews_per_movie:
            try:
                # Find the "load more" button on the webpage
                load_more = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@class="ipl-load-more__button"]'))
                )

                # Click on the "load more" button
                load_more.click()
                page += 1  # Move on to the next "load more" button
            except:
                # If no more "load more" button is found, stop
                break

            #  collect data from the entire website
            reviews = driver.find_elements_by_class_name('review-container')

            for review in reviews:
                try:
                    # Find the review title, rating, date, and user name for each review
                    rating = review.find_element_by_class_name('rating-other-user-rating').text
                    user_name = review.find_element_by_class_name('display-name-link').text

                    # Append the data to lists
                    collected_data["Movie_Title"].append(movie_title)
                    collected_data["Rating"].append(rating)
                    collected_data["User_Name"].append(user_name)

                    # Increment the collected reviews counter
                    collected_reviews += 1

                    if collected_reviews >= total_reviews_per_movie:
                        break  # Exit the loop if the desired number of reviews is reached

                except:
                    continue

    # Close the WebDriver
    driver.quit()

    return pd.DataFrame(collected_data)

In [ ]:
# So let's apply the function behing to movie_base and create a new dataframe 
collected_reviewers_df = collect_reviewers_for_movies(df, total_reviews_per_movie=10)

In [ ]:
# Save the dataFrame to a CSV file
collected_reviewers_df=pd.read_csv('rating.csv')